# This is just a notebook to practice getting live temperature data

In [13]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": -77.846,
	"longitude": 166.676,
	"minutely_15": "temperature_2m",
	"timezone": "GMT",
	"past_minutely_15": 24,
	"forecast_days": 1
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process minutely_15 data. The order of variables needs to be the same as requested.
minutely_15 = response.Minutely15()
minutely_15_temperature_2m = minutely_15.Variables(0).ValuesAsNumpy()

minutely_15_data = {"date": pd.date_range(
	start = pd.to_datetime(minutely_15.Time(), unit = "s", utc = True),
	end = pd.to_datetime(minutely_15.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = minutely_15.Interval()),
	inclusive = "left"
)}
minutely_15_data["temperature_2m"] = minutely_15_temperature_2m

minutely_15_dataframe = pd.DataFrame(data = minutely_15_data)
print(minutely_15_dataframe)


Coordinates -77.875°N 166.625°E
Elevation 47.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                         date  temperature_2m
0   2024-11-05 06:15:00+00:00      -16.250000
1   2024-11-05 06:30:00+00:00      -16.200001
2   2024-11-05 06:45:00+00:00      -16.150000
3   2024-11-05 07:00:00+00:00      -16.100000
4   2024-11-05 07:15:00+00:00      -16.100000
..                        ...             ...
307 2024-11-08 11:00:00+00:00      -13.250000
308 2024-11-08 11:15:00+00:00      -13.100000
309 2024-11-08 11:30:00+00:00      -12.900000
310 2024-11-08 11:45:00+00:00      -12.750000
311 2024-11-08 12:00:00+00:00      -12.600000

[312 rows x 2 columns]
